# Groupby, split-apply-combine and pandas

Netflix recently released some user ratings data. I wanted to ask a straightforward question: _do Netflix subscribers prefer older or newer movies?_

Intuitively, you want to split the dataset into groups, one for each year, and then to compute a summary statistic, such as the mean or the median, and then to see whether this statistic increases over the years (after this, you may want to perform a statistical test).

The great thing is that there is a conceptual framework for doing and thinking about this, along with implementations in both Python and R. The framework is known as split-apply-combine because we

* [Step 1](#step_1): split the data into groups by creating a _groupby_ object from the original DataFrame;
* [Step 2](#step_2): apply a function, in this case, an aggregation function that computes a summary statistic (you can also transform or filter your data in this step);
* [Step 3](#step_3): combine the results into a new DataFrame.

This is the conceptual framework for the analysis at hand. In this post you'll learn how to do this to answer the Netflix ratings question above using the Python package `pandas`. You could do the same in R using, for example, the `dplyr` package. I'll also necessarily delve into _groupby_ objects, wich are not the most intuitive objects. The process of split-apply-combine with _groupby_ objects is a pattern that we all perform intuitively, as we'll see, but it took took Hadley Wickham to formalize the procedure in 2011 with his paper _The Split-Apply-Combine Strategy for Data Analysis_.

If you find this technique useful, you can learn more about it (among many other things) and practice it in our [Manipulating DataFrames with pandas course](https://www.datacamp.com/courses/manipulating-dataframes-with-pandas), taught by Dhavide Aruliah of Anaconda.

## Data Exploration with pandas

### Import your data

Here you'll use `pandas`, _groupby_ objects and the principles of split-apply-combine to check out how Netflix movie ranges vary as a function of the year they were released. I originally came across the data on data.world [here](https://data.world/chasewillden/netflix-shows) and you can also find it at [here](https://theconceptcenter.com/simple-research-study-netflix-shows-analysis/) at The Concept Centre. You can find all the code in this post [here](https://github.com/datacamp/community-groupby) if you would like to reproduce it.

You'll first import the necessary packages and the data and check out the first five rows of the data:

In [ ]:
# Import packages and set visualization style
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [ ]:
# Import data and check out head of DataFrame
df = pd.read_csv('../data/netflix.csv')
df.head()

This looks pretty cool to me: you have titles, ratings, release year and user rating score, among several other columns. Before performing our _groupby_ and split-apply-combine procedure, lets look a bit more closely at the data to make sure it's what we think it is and to deal with missing values. Note that there is a missing value NaN in the *user_rating_score* of the second row (row 1).

### Summarising your data with plots and statistics

The `pandas` DataFrame `.info()` method is invaluable. Applying it below shows that you have 1000 rows and 7 columns of data, but also that the column of interest, *user_rating_score*, has only 605 non-null values. This means that there are 395 missing values:

In [ ]:
# Check out info of DataFrame
df.info()

You can drop rows that have any missing values, drop any duplicate rows and build a _pairplot_ of the DataFrame using `seaborn` in order to get a visual sense of the data. You'll color the data by the 'rating' column. Check out the plots and see what information you can get from them.

In [ ]:
# Drop rows with missing values and drop duplicate
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Visualize pairplot of df
sns.pairplot(df, hue='rating');

Check out, for example, *user_rating_score* as a function of *release_year*. There is *not* a visually identifiable trend but perhaps some data analysis will draw out any trends. If you'd like to check out several summary statistics of the DataFrame, you can also do this using the `.describe()` method:

In [ ]:
# Get summary stats of df
df.describe()

## Groupbys and split-apply-combine to answer the question

<a id='step_1'></a>

### Step 1. Split

Now that you've checked out out data, it's time for the fun part. You'll first use a *groupby* method to split the data into groups, where each group is the set of movies released in a given year. This is the *split* in split-apply-combine: 

In [ ]:
# Group by year
df_by_year = df.groupby('release_year')

This creates a *groupby* object:

In [ ]:
# Check type of GroupBy object
type(df_by_year)

<a id='step_2'></a>

### Step 2. Apply

Such *groupby* objects are very useful. Remember that the `.describe()` method for a DataFrame returns summary statistics for numeric columns? Well, the `.describe()` method for `DataFrameGroupBy` objects returns summary statistics for each numeric column, but computed for each group in the split. In your case, it's for each *release_year*. This is an example of the *apply* in split-apply-combine: you're *applying* the `.describe()` method to each group in the *groupby*. Do this and print the first 5 rows of the result:

In [ ]:
# Summary stats over years
df_by_year.describe().head()

If you want to see what the grouping looks like, you can pass the _groupby_ object to the function `list()`:

In [ ]:
# Cast grouping as a list and check out one year
list(df_by_year)[10]

<a id='step_3'></a>

### Step 3. Combine

Let's say that you wanted the mean or median *user_rating_score* for each year. Then you can apply the `.mean()` or `.median()` method, respectively, to the *groupby* object and 'combine' these into a new DataFrame.

In [ ]:
# Get median values by year and print first 5 rows
df_med_by_year = df_by_year.median()
df_med_by_year.head()

There's an important subtlety concerning the index of the DataFrame `df_med_by_year`. Recall that the index of a DataFrame consists of the row labels. Check out the index of the original DataFrame `df`:

In [ ]:
# Print index of df
print(df.index)

This index consists of  the original row numbers, labelled by integers. '1' is missing as you dropped some rows above. The index of `df_med_by_year` consists of the values in the original column that you grouped by, the years from *release_year*:

In [ ]:
# Print index
print(df_med_by_year.index)

You're interested in the *user_rating_score* column, which contains the median rating for each year. You can slice out the *user_rating_score* column of `df_med_by_year`  and plot it as a function of the year (given by the index of the DataFrame `df_rat_by_year`):

In [ ]:
# Slice out user rating and plot
df_rat_by_year = df_med_by_year['user_rating_score']
plt.scatter(df_rat_by_year.index, df_rat_by_year)
plt.xlabel('year of release')
plt.ylabel('median rating');

Looking at the figure, the median rating definitely increases over time. You'd need to leverage some more sophisticated statistics to convince me of the trend in general but this an example of Exploratory Data Analysis being a great starting point for further research. 

## Groupbys and split-apply-combine in daily use

*Groupby* objects are not intuitive. They do, however, correspond to a natural the act of splitting a dataset with respect to one its columns (or more than one, but let's save that for another post about grouping by multiple columns and hierarchical indexes).

The split-apply-combine principle is not only elegant and practical, it's something that Data Scientists use daily, as in the above example. To appreciate more of its uses, check out Hadley Wickham's original [paper](https://www.jstatsoft.org/article/view/v040i01) *The Split-Apply-Combine Strategy for Data Analysis*. If you have any thoughts, responses and/or ruminations, feel free to reach out to me on twitter: [@hugobowne](https://twitter.com/hugobowne).